<a href="https://colab.research.google.com/github/Vakhranev/Compling/blob/master/Vakhranyov_AY_Deep_learning_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом проекте я постарался с помощью предобученного берта сделать предсказания для текста на сентимент-анализ. Я взял датасет Sentiment Analysis Dataset.csv он есть здесь (https://github.com/vineetdhanawat/twitter-sentiment-analysis/blob/master/datasets/Sentiment%20Analysis%20Dataset.csv). Пояснений к нему, к сожалению, нет, поэтому точно сказать не могу, но, судя по всему, представленные в нём разбиты следующим образом: целевой класс (представлены единицей) — твиты с положительно-окрашенным мнением, всё остальное — представлено нулями. Помимо того, что я попробовал замерять лоссы на эпохах, также я использовал ф-меру для оценки модели.

В последнее время применение глубокого обучения для решения проблемы сентимент-анализа стало популярной темой исследований. Существуют различные архитектуры глубокого обучения и технологии, которые применяют для подобного рода анализа: эмбеддинги, автоэнкодеры, CNN, RNN, LSTM, применение аттеншн-механизма в RNN, MemNN, RecNN. Многие из этих методов глубокого обучения показали отличные результаты для различных задач сентимент-анализа. Если верить научным работам по теме, которые я просмотрел, то с развитием исследований и приложений глубокого обучения в ближайшем будущем появятся более интересные исследования применения глубокого обучения для сентимент-анализа.

In [16]:
!pip install transformers
import pandas as pd
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
data = pd.read_csv("Sentiment Analysis Dataset.csv", header= None)
max_len = 512

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
data = data[:5000]
data.columns = ["id", "sentiment", "text"]

In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["id"] = label_encoder.fit_transform(data["id"])
data["sentiment"] = label_encoder.fit_transform(data["sentiment"])
data["text"] = label_encoder.fit_transform(data["text"])

In [19]:
data.head

<bound method NDFrame.head of         id  sentiment  text
0     4999          2  4999
1        0          0     2
2     1111          0     3
3     2222          1     6
4     3333          0     9
...    ...        ...   ...
4995  4439          0  4155
4996  4440          0  4156
4997  4441          1  4157
4998  4442          0  4158
4999  4446          0  1935

[5000 rows x 3 columns]>

In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [21]:
from torch.utils.data import Dataset
class dataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):   
        text = str(self.texts[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(text, add_special_tokens=True,max_length=self.max_len, return_token_type_ids=False,padding='max_length', return_attention_mask=True, return_tensors='pt', truncation = True)
        return {'text': text, 'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'targets': torch.tensor(target, dtype=torch.long)}

In [22]:
import numpy as np
from torch.utils.data import DataLoader
def fn_Dataloader(data, tokenizer, max_len, batch_size):
    ds = dataset(data.text.to_numpy(), data.sentiment.to_numpy(), tokenizer=tokenizer, max_len=max_len)
    return DataLoader(ds,batch_size=batch_size)

batch_size = 8
loader = fn_Dataloader(train, tokenizer, max_len, batch_size)
test_loader = fn_Dataloader(test, tokenizer, max_len, batch_size)

In [23]:
print(type(loader))

torch.utils.data.dataloader.DataLoader


In [24]:
from transformers import BertModel
model = BertModel.from_pretrained('bert-base-uncased', return_dict=False)

In [25]:
import torch
from torch import nn
class Classifier(torch.nn.Module):
    
    def __init__(self,n_classes):
        super(Classifier, self).__init__()
        self.bert = model
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Linear(32, n_classes)
        )
    
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        output = self.drop(pooled_output)
        return self.out(output)

In [26]:
model = Classifier(3)

In [27]:
epochs = 6
for param in model.parameters():
    param.requires_grad = False
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from tqdm import tqdm
from sklearn.metrics import f1_score
from statistics import mean
train_losses = list()
train_loss_per_epoch = list()
test_loss_per_epoch = list()
for epoch in range(epochs):
    train_epoch_losses = list()
    test_epoch_losses = list()
    train_f1s = list()
    test_f1s = list()
    print(f'Epoch: {epoch}')
    model.train()
    for element in tqdm(loader):
        x = element["input_ids"]
        mask_train = element["attention_mask"]
        y = element["targets"]
        outputs = model(input_ids=x,attention_mask=mask_train)
        _, preds = torch.max(outputs, dim=1)
        train_loss = loss(outputs, y)
        optimizer.step()
        optimizer.zero_grad()
        train_losses.append(train_loss.item())
        train_epoch_losses.append(train_loss.item())
        train_f1s.append(f1_score(y, outputs.argmax(1), average='micro'))
    model.eval()
    for element in tqdm(test_loader):
        x = element["input_ids"]
        mask_test = element["attention_mask"]
        y = element["targets"]
        with torch.no_grad():
            outputs_test = model(input_ids=x,attention_mask=mask_test)
            _, preds_test = torch.max(outputs_test, dim=1)
            test_loss = loss(outputs_test, y)
        test_epoch_losses.append(test_loss.item())
    train_loss_per_epoch.append(np.mean(train_epoch_losses))
    test_loss_per_epoch.append(np.mean(test_epoch_losses))
    test_f1s.append(f1_score(y, outputs_test.argmax(1), average='micro'))
    mean_train = mean(train_f1s)
    mean_test = mean(test_f1s)
    print(f'Epoch {epoch}')
    print(f'Loss: train {train_loss_per_epoch[-1]:.2f} | test {test_loss_per_epoch[-1]:.2f}')
    print(f'F1: train {mean_train} | test {mean_test}')


  0%|          | 0/563 [00:00<?, ?it/s]

Epoch: 0



100%|██████████| 563/563 [2:24:39<00:00, 15.42s/it]

100%|██████████| 63/63 [14:44<00:00, 14.05s/it]

  0%|          | 0/563 [00:00<?, ?it/s]

Epoch 0
Loss: train 1.11 | test 1.08
F1: train 0.35324156305506216 | test 0.5
Epoch: 1



100%|██████████| 563/563 [2:25:28<00:00, 15.50s/it]

100%|██████████| 63/63 [14:28<00:00, 13.79s/it]

  0%|          | 0/563 [00:00<?, ?it/s]

...


100%|██████████| 563/563 [2:25:23<00:00, 15.49s/it]

100%|██████████| 63/63 [14:34<00:00, 13.88s/it]

  0%|          | 0/563 [00:00<?, ?it/s]

Epoch 2
Loss: train 1.11 | test 1.13
F1: train 0.34369449378330375 | test 0.5
Epoch: 3



100%|██████████| 563/563 [2:25:28<00:00, 15.50s/it]

100%|██████████| 63/63 [14:25<00:00, 13.74s/it]

  0%|          | 0/563 [00:00<?, ?it/s]

Epoch 3
Loss: train 1.11 | test 1.12
F1: train 0.34436056838365897 | test 0.5
Epoch: 4



  2%|▏         | 13/563 [03:20<2:22:04, 15.50s/it]

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(train_loss_per_epoch)
plt.grid()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(test_loss_per_epoch)
plt.grid()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(14, 12))
plt.plot(train_f1s)
plt.grid()

In [ ]:
print('Лучшая метрика на test:')
print('Loss: ', min(test_loss_per_epoch))

Я выдающимися знаниями особо никогда не блистал, поэтому для меня даже достаточно простая задача (если сравнивать с другими вариантами проектов) давалась непросто. Наверное, главная проблема, с которой я столкнулся — это подсчёт ф-меры на тесте. На любом количестве — даже на сотнях и десятках тысяч я упорно получал нули после каждой эпохи. В причнах этой проблемы, к сожалению, мне разобраться не удалось. Полагаю, что это может быть связано с не самой очевидной разметкой данных в датасете. Я пробовал увеличивать количество эпох обучения, но результат не менялся. Уменьшать learning rate я не стал, потому что по умолчанию он и так стоит небольшой, вряд ли это помогло бы с решением проблемы.
Ф-мера на трейне вела себя довольно хаотично. Также, как видно на графиках, на 6 (последней) эпохе резко портятся лоссы (особенно на тесте), что, вероятнее всего, свидетельствует о переобучении модели. Думаю, что на таком количестве данных оптимально было бы остановиться на 5 эпохах.